**Access Data**

In [ ]:
application_id = "application-id"
directory_id = "directory-id"
secret = "secret"

In [ ]:
spark.conf.set("fs.azure.account.auth.type.storage.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.storage.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.storage.dfs.core.windows.net", f"{application_id}")
spark.conf.set("fs.azure.account.oauth2.client.secret.storage.dfs.core.windows.net", f"{secret}")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.storage.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")


**Make Database**

In [ ]:
-- %sql
DROP DATABASE IF EXISTS gold_db;
CREATE DATABASE gold_db;

# Read/Write Data; Make Delta Tables

**Import + store folder paths**

In [ ]:
from pyspark.sql.functions import *
silver_path = "abfss://silver@storage.dfs.core.windows.net"
gold_path = "abfss://gold@storage.dfs.core.windows.net"

**Zone Type**

In [ ]:
trip_zone_df = spark.read.format("parquet").option("inferSchema", True).option("header", True)\
    .load(f"{silver_path}/trip_zone")
trip_zone_df.display()
trip_zone_df.write.format("delta").mode("append").option("path", f"{gold_path}/trip_zone")\
    .saveAsTable("gold_db.trip_zone")

**Trip Type**

In [ ]:
trip_type_df = spark.read.format("parquet").option("inferSchema", True).option("header", True)\
    .load(f"{silver_path}/trip_type")
trip_zone_df.display()
trip_type_df.write.format("delta").mode("overwrite").option("path", f"{gold_path}/trip_type")\
    .saveAsTable("gold_db.trip_type")

**Trip Data**

In [ ]:
trip_df = spark.read.format("parquet").option("inferSchema", True).option("header", True)\
    .load(f"{silver_path}/trips")
trip_df.display()
trip_df.write.format("delta").mode("overwrite").option("path", f"{gold_path}/trips").saveAsTable("gold_db.trips")

**Run commands to test on database, then restore back to original**

Change Borough column where Location ID is 1

In [ ]:
-- %sql
UPDATE gold_db.trip_zone SET Borough = "EMR" WHERE LocationID = 1;
SELECT * FROM gold_db.trip_zone;

Delete row where LocationID is 1 and show that selecting it returns nothing

In [ ]:
-- %sql
DELETE FROM gold_db.trip_zone WHERE LocationID = 1;
SELECT * FROM gold_db.trip_zone WHERE LocationID = 1;

Reset database back to original copy

In [ ]:

-- %sql
RESTORE gold_db.trip_zone TO VERSION AS OF 0;

# Delta Tables

In [ ]:
-- %sql
SELECT * FROM gold_db.trip_type

In [ ]:
-- %sql
SELECT * FROM gold_db.trip_zone

In [ ]:
-- %sql
SELECT * FROM gold_db.trips